# The Battle of Neighborhoods

## 1. Introduction
Discuss the business problem and audience who would interested in this project.

### Business Problem: Recommend the *best place* to open a *Cafe* in Singapore.

The Cafe will serve *pastries and good coffee*. Seatings will also be provided for visitors to enjoy their coffee and meet up with friends. People will visit Cafe for numerous reasons and on different occasions. On *weekdays*, students might visit after school to *do assignments* together with their school mates. For cafes near to offices, working adults may use it as a place for *informal meetings and discussions* or just to *grab a cup of coffee* in the morning before heading to the office. On *weekends*, adults will usually visit a Cafe to *meet and catch up with friends*. For cafes near to *public transportations* and *attractions or hostels and hotels*, tourists will tend to visit due to *convenience*. Therefore, we can infer firsthand that the visitors to the cafe is *heavily dependent on its location on weekdays* but less likely on weekends.

In order to maximise revenues, cafes need to be located near offices to capture the weekday crowd but also not too far away from places of attractions and hotels where tourists will pay a visit. As working adults has larger spending powers than students, the analysis will focus more on the working adults.

## 2. Data
Describe the data that you will be using to solve the problem or execute your idea. 

### 1. Singapore City data that contains Towns, Neighborhoods along with their latitude and longitude.

   (Data Source:  Foursquare API)
   
   (Data Source:  Wikipedia: https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore)

The *5* Regions of Singapore
1. Central Region
2. East Region
3. North-East Region
4. North Region
5. West Region

There are a total of *55* planning areas organised into these 5 regions. The information of these 55 planning areas can be extracted from wikipedia.


Tanjong Pagar is a multicultural district in Singapore with international dining and diverse nightlife options. Craig Road and Duxton Hill have a mix of bars and global restaurants, while Maxwell Road Food Centre is known for street food such as noodles and chicken rice. “Little Korea”, at the northern end of Tanjong Pagar Road, has casual eateries and specialist grocery stores. Nearby, Everton Park is known for its hip cafes. Therefore Tanjong Pagar is being selected to be the district to study.
   
   

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Webscrap Singapore's Neighborhood from Wikipedia

Then transform the data into a pandas dataframe that consists of five columns: Name, Region, Area, Population and Density

In [2]:
from bs4 import BeautifulSoup

source = requests.get('https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore').text

soup = BeautifulSoup(source, 'html5lib')

In [10]:
table_post = soup.find_all('table')[2] #Grab the third table
fields = table_post.find_all('td') 

EName = []
MName =[]
CName=[]
Pinyin =[]
TName =[]
Region = []
Area =[]
Population = []
Density =[]

for i in range(0, len(fields), 9):
    EName.append(fields[i].text.strip())
    MName.append(fields[i+1].text.strip())
    CName.append(fields[i+2].text.strip())
    Pinyin.append(fields[i+3].text.strip())
    TName.append(fields[i+4].text.strip())
    Region.append(fields[i+5].text.strip())
    Area.append(fields[i+6].text.strip())
    Population.append(fields[i+7].text.strip())
    Density.append(fields[i+8].text.strip())

df_pc = pd.DataFrame(data=[EName, MName, CName, Pinyin, TName, Region, Area, Population, Density]).transpose()
df_pc.columns = ['EName', 'MName', 'CName','Pinyin','TName','Region','Area','Population','Density']
df_pc.head()

,EName,MName,CName,Pinyin,TName,Region,Area,Population,Density
0,Ang Mo Kio,,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,"165,710","12,000"
1,Bedok,*,勿洛,Wù luò,பிடோ,East,21.69,"281,300","13,000"
2,Bishan,,碧山,Bì shān,பீஷான்,Central,7.62,"88,490","12,000"
3,Boon Lay,,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,"144,410","13,000"


### Define Foursquare Credentials and Version

In [11]:
# The code was removed by Watson Studio for sharing.

### Get Singapore Geographical Coordinates

In [5]:
address = 'Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.3408528, 103.878446863736.


In [73]:
search_query = 'Cafe'
radius = 5000
print(search_query + ' .... OK!')

Cafe .... OK!


In [74]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UCVO2GVTL12OH0OWGRITBRVBUO2XABHYHFTVA2YIYXWZY2XE&client_secret=C5HRVGKZXEGWYOV0D03E4HRRCM5JW2AJ44A3ORJXDLSFLFUO&ll=1.2764189,103.8429295&v=20180604&query=Cafe&radius=5000&limit=30'

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ceabcd94434b92169576f9d'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'primary': True,
      'shortName': 'Café'}],
    'hasPerk': False,
    'id': '52bd3837498e63a584baa251',
    'location': {'address': '202 Telok Ayer Street',
     'cc': 'SG',
     'city': 'Singapore',
     'country': 'Singapore',
     'distance': 631,
     'formattedAddress': ['202 Telok Ayer Street', '068639', 'Singapore'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 1.28009,
       'lng': 103.847253}],
     'lat': 1.28009,
     'lng': 103.847253,
     'postalCode': '068639'},
    'name': 'My Awesome Cafe 真棒堂 (My Awesome Café)',
    'referralId': 'v-1558887641'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix':

In [76]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,name,referralId
0,"[{'pluralName': 'Cafés', 'shortName': 'Café', ...",False,52bd3837498e63a584baa251,202 Telok Ayer Street,SG,Singapore,Singapore,NaN,631,"[202 Telok Ayer Street, 068639, Singapore]","[{'lat': 1.28009, 'label': 'display', 'lng': 1...",1.280090,103.847253,NaN,068639,My Awesome Cafe 真棒堂 (My Awesome Café),v-1558887641
1,"[{'pluralName': 'American Restaurants', 'short...",False,4b610addf964a52081072ae3,"Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong...",SG,Singapore,Singapore,NaN,82,"[Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjon...","[{'lat': 1.2768220777373407, 'label': 'display...",1.276822,103.843549,NaN,081007,Bluemist Cafe Bistro,v-1558887641
2,"[{'pluralName': 'Nightclubs', 'shortName': 'Ni...",False,4b0d52c7f964a520504623e3,65 Neil Rd.,SG,Singapore,Singapore,NaN,342,"[65 Neil Rd., 088897, Singapore]","[{'lat': 1.2794357645685264, 'label': 'display...",1.279436,103.842319,NaN,088897,Taboo Café & Bar,v-1558887641
3,"[{'pluralName': 'Asian Restaurants', 'shortNam...",False,4d58bc313281b1f7b468b32f,7 Craig Rd.,SG,Singapore,Singapore,NaN,118,"[7 Craig Rd., 089667, Singapore]","[{'lat': 1.2774705411789302, 'label': 'display...",1.277471,103.843107,NaN,089667,Golden Cafe Eating House,v-1558887641
4,"[{'pluralName': 'Restaurants', 'shortName': 'R...",False,4b84e425f964a520c64631e3,114 Tanjong Pagar Road,SG,Singapore,Singapore,NaN,177,"[114 Tanjong Pagar Road, 088529, Singapore]","[{'lat': 1.2778405205280887, 'label': 'display...",1.277841,103.843649,NaN,088529,New Harbour Cafe & Bar,v-1558887641


In [77]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,id
0,My Awesome Cafe 真棒堂 (My Awesome Café),Café,202 Telok Ayer Street,SG,Singapore,Singapore,NaN,631,"[202 Telok Ayer Street, 068639, Singapore]","[{'lat': 1.28009, 'label': 'display', 'lng': 1...",1.280090,103.847253,NaN,068639,52bd3837498e63a584baa251
1,Bluemist Cafe Bistro,American Restaurant,"Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong...",SG,Singapore,Singapore,NaN,82,"[Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjon...","[{'lat': 1.2768220777373407, 'label': 'display...",1.276822,103.843549,NaN,081007,4b610addf964a52081072ae3
2,Taboo Café & Bar,Nightclub,65 Neil Rd.,SG,Singapore,Singapore,NaN,342,"[65 Neil Rd., 088897, Singapore]","[{'lat': 1.2794357645685264, 'label': 'display...",1.279436,103.842319,NaN,088897,4b0d52c7f964a520504623e3
3,Golden Cafe Eating House,Asian Restaurant,7 Craig Rd.,SG,Singapore,Singapore,NaN,118,"[7 Craig Rd., 089667, Singapore]","[{'lat': 1.2774705411789302, 'label': 'display...",1.277471,103.843107,NaN,089667,4d58bc313281b1f7b468b32f
4,New Harbour Cafe & Bar,Restaurant,114 Tanjong Pagar Road,SG,Singapore,Singapore,NaN,177,"[114 Tanjong Pagar Road, 088529, Singapore]","[{'lat': 1.2778405205280887, 'label': 'display...",1.277841,103.843649,NaN,088529,4b84e425f964a520c64631e3
5,Cafe 2000,Café,M Hotel,SG,Singapore,Singapore,81 Anson Rd.,347,"[M Hotel (81 Anson Rd.), 079908, Singapore]","[{'lat': 1.2739816398549644, 'label': 'display...",1.273982,103.844885,NaN,079908,4b987225f964a520954235e3
6,Han's Cafe - Tanjong Pagar Center,Café,7 Wallich Street,SG,NaN,Singapore,NaN,236,"[7 Wallich Street, 078879, Singapore]","[{'lat': 1.2767379869213256, 'label': 'display...",1.276738,103.845032,NaN,078879,58575b058f0be459e4487256
7,Imperial Treasure Bakery & Cafe 御宝面包之窗,Bakery,#01-10/14A 100AM Mall,SG,Singapore,Singapore,100 Tras St,188,"[#01-10/14A 100AM Mall (100 Tras St), 079027, ...","[{'lat': 1.274762105690608, 'label': 'display'...",1.274762,103.843292,Financial District,079027,50d115bfe4b072dcb75a475b
8,McDonald's / McCafé,Fast Food Restaurant,"#01-03, Springleaf Tower",SG,Singapore,Singapore,3 Anson Rd,378,"[#01-03, Springleaf Tower (3 Anson Rd), 079909...","[{'lat': 1.2748651681501353, 'label': 'display...",1.274865,103.845952,Financial District,079909,4b57df11f964a520984428e3
9,Cuscaden Cafe & Pub,Pub,18 Duxton Rd,SG,Singapore,Singapore,NaN,349,"[18 Duxton Rd, Singapore]","[{'lat': 1.2795303917866119, 'label': 'display...",1.279530,103.843319,NaN,NaN,4d1b587883c5a1cdc5132b4e


In [56]:
dataframe_filtered.name

0            Xin Wang Hong Kong Café
1                      Graffiti Cafe
2                 Ya Kun Family Cafe
3                           KOI Café
4            Hot Tomato Cafe & Grill
5                           Cafe 107
6                       Mustafa Cafe
7                      Ice Edge Cafe
8                      Mad Jack Café
9     Hatter Street Bakehouse & Café
10          Hard Rock Cafe Singapore
11                       Cafe @ MSHS
12                     Tea Tree Cafe
13                      Revzone Cafe
14                        Mac's Cafe
15                 Maris Stella Cafe
16                   Joan Bowen Cafe
17        Swee Heng 1989 Bakery Café
18          HOT SPOT Cafe Restaurant
19                     Cafe Football
20                 Kampong Glam Cafe
21                 High Society Café
22           Xin Wang Hong Kong Café
23                  Bulldog Cafe Pub
24                          KOI Café
25           Han's Café & Cake House
26                          PS. Cafe
2

In [81]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Center point of Tanjong Pagar
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Tanjong Pagar',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Cafes as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [3]:
# install the google map api client library
!pip install -U googlemaps

Requirement not upgraded as not directly required: requests<3.0,>=2.11.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from googlemaps) (2.18.4)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests<3.0,>=2.11.1->googlemaps) (3.0.4)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests<3.0,>=2.11.1->googlemaps) (2.6)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests<3.0,>=2.11.1->googlemaps) (1.22)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests<3.0,>=2.11.1->googlemaps) (2019.3.9)
  Running setup.py bdist_wheel for googlemaps ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/3c/3f/25/ce6

In [4]:
google_act = None
with open('google_map_act.json', 'r') as f:
    google_act = json.load(f)
    
GOOGLE_MAP_API_KEY = google_act['api_key']    

import googlemaps
gmaps = googlemaps.Client(key=GOOGLE_MAP_API_KEY)

FileNotFoundError: [Errno 2] No such file or directory: 'google_map_act.json'